# Import

In [51]:
import sys
import os
# 標準エラー出力を/dev/nullにリダイレクト
sys.stderr = open(os.devnull, 'w')

import pandas as pd
import numpy as np
import japanize_matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from geopy.distance import geodesic
import math
import folium
import ast
from sklearn.preprocessing import StandardScaler

# データの読み込み

In [52]:
gad_df = pd.read_csv(r"C:\Users\chaki\Desktop\jupyter_file\data\2024_data\GAD_sapporo_Nov.csv")

In [54]:
gad_df

,agent_id,dep_time,dep_time_tz,arr_time,arr_time_tz,activity_time,next_dep_time,next_dep_time_tz,trav_time,wait_time,...,age,gender,home_lon,home_lat,work_lon,work_lat,geometry,短縮名称,trav_time_timedelta,trav_time_minutes
0,673566,10:15:00,10:15:00+09,10:23:03,10:23:03+09,01:49:57,12:13:00,12:13:00+09,00:08:03,00:00:00,...,60,1,141.322249,43.101292,NaN,NaN,POINT (-71633.39451479971 -102719.67518559382),札幌市,0 days 00:08:03,8.0
1,673566,12:13:00,12:13:00+09,12:21:14,12:21:14+09,NaN,NaN,NaN,00:08:14,00:00:00,...,60,1,141.322249,43.101292,NaN,NaN,POINT (-75518.00310670525 -99421.77035360345),札幌市,0 days 00:08:14,8.0
2,673571,07:40:00,07:40:00+09,07:47:30,07:47:30+09,01:58:30,09:46:00,09:46:00+09,00:07:30,00:00:00,...,60,1,141.321888,43.101515,NaN,NaN,POINT (-76213.30828972792 -97874.49461021337),札幌市,0 days 00:07:30,8.0
3,673571,09:46:00,09:46:00+09,09:55:46,09:55:46+09,00:52:14,10:48:00,10:48:00+09,00:09:46,00:00:00,...,60,1,141.321888,43.101515,NaN,NaN,POINT (-73929.34033987307 -100720.72224771477),札幌市,0 days 00:09:46,10.0
4,673571,10:48:00,10:48:00+09,10:55:19,10:55:19+09,01:55:41,12:51:00,12:51:00+09,00:07:19,00:00:00,...,60,1,141.321888,43.101515,NaN,NaN,POINT (-75905.87475582902 -104927.67108446534),札幌市,0 days 00:07:19,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2939981,1219954,15:40:00,15:40:00+09,15:42:05,15:42:05+09,01:56:55,17:39:00,17:39:00+09,00:02:05,00:00:00,...,20,2,141.395393,43.022432,141.329832,43.063847,POINT (-69653.31130175502 -108245.16061833124),札幌市,0 days 00:02:05,2.0
2939982,1219954,17:39:00,17:39:00+09,17:45:53,17:45:53+09,03:59:07,21:45:00,21:45:00+09,00:06:53,00:00:00,...,20,2,141.395393,43.022432,141.329832,43.063847,POINT (-73145.52509013907 -104484.95068646719),札幌市,0 days 00:06:53,7.0
2939983,1219954,21:45:00,21:45:00+09,21:51:49,21:51:49+09,NaN,NaN,NaN,00:06:49,00:00:00,...,20,2,141.395393,43.022432,141.329832,43.063847,POINT (-69653.31130175502 -108245.16061833124),札幌市,0 days 00:06:49,7.0
2939984,1219958,12:23:00,12:23:00+09,12:25:20,12:25:20+09,08:58:40,21:24:00,21:24:00+09,00:02:20,00:00:00,...,20,2,141.344092,43.055373,141.352621,43.052294,POINT (-73103.95362709576 -104891.56731164882),札幌市,0 days 00:02:20,2.0


In [55]:
gad_df = gad_df.rename(columns={"end_activity_type":"purpose"})
gad_df = gad_df.rename(columns={"longest_distance_mode":"mode"})
gad_df['trav_time_timedelta'] = pd.to_timedelta(gad_df['trav_time'])
# timedeltaから分のみ（総分数）を取得
gad_df['trav_time_minutes'] = round(gad_df['trav_time_timedelta'].dt.total_seconds() / 60)
gad_df = gad_df.dropna(subset=['trav_time'])
gad_df = gad_df.drop(['dep_time_tz', 'arr_time_tz', 'next_dep_time_tz', 'wait_time', 'euclidean_distance', 'start_activity_type_id', 'end_activity_type_id', 'start_key_code', 'end_key_code', 'modes_id', 'work_lon', 'work_lat', '短縮名称'], axis=1)


In [56]:
gad_df['judge'] = (gad_df['mode'] == 'car') & (gad_df['trav_time_minutes'] <= 5) & (gad_df['traveled_distance'] <= 2000) | (gad_df['mode'] == 'walk') & (gad_df['traveled_distance'] >= 1200)

# 全体代替可能トリップチェーン抽出

In [57]:
gad_walk = gad_df[gad_df['mode'].isin(['walk'])].reset_index(drop=True)
gad_car = gad_df[gad_df['mode'].isin(['car'])].reset_index(drop=True)
short_car =  gad_car[(gad_car['trav_time_minutes'] <= 5) & (gad_car['traveled_distance'] <= 2000)].reset_index(drop=True)
long_walk = gad_walk[gad_walk['traveled_distance'] >= 1200].reset_index(drop=True)
agent_df = pd.concat([short_car, long_walk], ignore_index=True)

In [58]:
# 代替可能性があるトリップのユーザーID
id_list = list(set(agent_df['agent_id']))

In [59]:
def check_all_rows_valid(df):
    return ((df['judge'] == True) | ((df['judge'] == False) & (df['mode'] == 'walk'))).all()

true_df = pd.DataFrame(columns=gad_df.columns)
false_df = pd.DataFrame(columns=gad_df.columns)
for i in range(len(id_list)):
    df = gad_df[gad_df['agent_id'].isin([id_list[i]])]
    if check_all_rows_valid(df):
        true_df = pd.concat([true_df, df], ignore_index=True)
    else:
        false_df = pd.concat([false_df, df], ignore_index=True)  


In [60]:
true_df['agent_id'].value_counts()

agent_id
54317      13
1367298    11
113878     11
1312434    10
1381127    10
           ..
1422134     2
1422152     2
1422155     2
1422165     2
2097148     2
Name: count, Length: 203184, dtype: int64

In [62]:
true_df.to_csv(r'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\data\全体代替可能data.csv')

## 分析 